## Rmax sequence prediction with IBTrACS

The goal is to accurately predict Rmax time series of any given TC given a set of parameters of interest from IBTrACS (say n = 8 parameters): Vmax, R34, lon, lat, etc...

We will compare several methods: sparse regression (Lasso), analog forecasting (KNN), and Multi-Layer-Perceptron (MLP). We will also test the Chavas and Knaff 2022 model as a reference baseline. Later, we will also use Data Assimilation to answer this challenge.

In [1]:
# General
# import glob
import os.path
# import warnings
# warnings.filterwarnings('ignore')
from tqdm import tqdm

# Arrays & Displays
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# from mpl_toolkits.axes_grid1 import make_axes_locatable
# # from matplotlib.colors import Normalize
# # from matplotlib.colors import ListedColormap
# # import matplotlib.cm as cm
# import pandas as pd

# PyTorch
import torch
import torch.nn as nn
from torch.autograd import Variable              # Convert arrays to tensors
from torch.utils.data import Dataset, DataLoader # Create a Dataset class to combine with DataLoader (= mini batches selection)
import pytorch_lightning as pl


# Data treatment
# import dask as da
# from dask.diagnostics import ProgressBar
# import zarr
# from scipy.interpolate import griddata
from datetime import datetime

# Custom
import dataUtils    as du
import pytorchUtils as pu

# Statistics
from sklearn import linear_model, neighbors

# Default parameters
mpl.rcParams.update({'font.size': 18})
mpl.rcParams['figure.figsize'] = (15, 10)
mpl.rcParams['axes.facecolor'] = 'white'
mpl.rcParams['figure.facecolor'] = 'white'

In [2]:
### Setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
print('{} GPU(s) available'.format(torch.cuda.device_count()))

Using cuda device
1 GPU(s) available


In [3]:
### VARIABLES TO CONSIDER
'''Initial dataset has 147 variables, so we select only a subset of these'''
# storm speed, time, dist2land, usa_r64, usa_r50
params_of_interest = ['usa_lon', 'usa_lat', 'usa_wind', 'usa_r34', 'usa_rmw'] 
input_variables    = ['usa_lon', 'usa_lat', 'usa_wind', 'usa_r34'] 
target_variable    = ['usa_rmw']
additional_info    = ['numobs', 'sid', 'basin', 'name', 'usa_agency', 'iso_time', 'usa_status']

### PARAMS
PARAMS = {'seq_len':     4, # length of the input time series used to predict y(t)
          'n_features':  len(params_of_interest),     # nb of output features
          
          # Model parameters
          'input_size':  5,     # nb of input features
          'hidden_size': 20,    # nb of features in hidden state
          'num_layers':  2,     # nb of stacked lstm layers
          'dropout':     0,   # dropout probability
    
          'batch_size':  8,     
          'n_epochs':    30,     # nb of epochs
          'learn_rate':  0.001, # learning rate
          
          'save_figs':       False,
          'feature_scaling': True,
         }

### PATHS
PATHS  = {
    # Data
    'ibtracs_data': '/home/arthur/data/ibtracs/IBTrACS.NA.v04r00.nc', # '/home/arthur/data/ibtracs/IBTrACS.NA.v04r00.nc'
    # Save
    'lstm_path':    '/home/arthur/results/TCsLifeMonitFromObs/rmax_seq_pred_ibtracsv/lstmv01.pth', 
}

In [4]:
### OPEN DATASET
ds_ibt_raw = xr.open_dataset(PATHS['ibtracs_data'])
ds_ibt     = ds_ibt_raw[params_of_interest + additional_info]
# ds_ibt_raw

In [5]:
ds_ibt_raw.isel(storm=25)['usa_rmw']

<xarray.DataArray 'usa_rmw' (date_time: 360)>
array([nan, nan, nan, ..., nan, nan, nan], dtype=float32)
Coordinates:
    time     (date_time) datetime64[ns] ...
    lat      (date_time) float32 ...
    lon      (date_time) float32 ...
Dimensions without coordinates: date_time
Attributes:
    long_name:              Radius of maximum winds (not best tracked)
    standard_name:          radius_of_tropical_cyclone_maximum_sustained_wind...
    units:                  nmile
    valid_min:              1
    valid_max:              1000
    coverage_content_type:  physicalMeasurement

In [6]:
### FILTERS
# By year
start_date = np.datetime64('2000-01-01')
fsi        = np.nanargmin(np.abs(ds_ibt['time'][:, 0] - start_date)) # First storm index
ds_ibt     = ds_ibt.isel(storm=slice(fsi, -1))
# By latitude
ds_ibt     = ds_ibt.where(np.abs(ds_ibt['lat']) <= 30)
# By removing empty Rmax time series
ds_ibt     = ds_ibt.where(ds_ibt['usa_rmw'].notnull().sum(axis=1) > 5)
# By removing empty R34 time series
ds_ibt     = ds_ibt.where(ds_ibt['usa_r34'].mean(dim='quadrant', skipna=True).notnull()) 
# By agency
ds_ibt     = ds_ibt.where(b'hurdat_atl' in ds_ibt['usa_agency'])
# By category
ds_ibt     = ds_ibt.where(ds_ibt['usa_wind'].max(dim='date_time', skipna=True) > 64) # >= Cat.1 according to Saffir Simpson scale, 64 is in knts


# First average on every quadrant
ds_ibt     = ds_ibt.mean(dim='quadrant', skipna=True)

# Keep only interesting time series, i.e containing sufficiently long sequences of valid Rmax values
for s in tqdm(range(len(ds_ibt['storm']))):
    da = ds_ibt.isel(storm=s)['usa_rmw']
    if np.max(np.diff(np.where(np.isnan(da)))) < 8: # Count maximum valid sequence and filter it out if less than 1 day
        ds_ibt.isel(storm=s)['usa_rmw'] *= np.nan
        
# Drop NaNs
ds_ibt = ds_ibt.dropna(dim='storm', how='all', subset=['usa_rmw']) # Much quicker to drop NaNs only at the end

# Convert to m/s and km units
ds_ibt['usa_wind'] *= 0.5144
ds_ibt['usa_r34']  *= 1.852
ds_ibt['usa_rmw']  *= 1.852

print("Total samples after filtering: ", len(ds_ibt['storm']))

# # Find a storm index by ID:
# x = ds_ibt_raw['sid'].where(ds_ibt_raw['sid'] == b'2017260N12310')
# np.argwhere(x.values==x.values)
# MARIA 2017: ds_ibt_raw.isel(storm=2199); ds_ibt.isel(storm=301)

100%|███████████████████████████████████████| 394/394 [00:00<00:00, 1104.28it/s]

Total samples after filtering:  115


In [7]:
### TODO: normalization, train valid test separation
# Follow noteboov 02***v02

### ZERO PADDING
for param in params_of_interest:
    ds_ibt[param] = ds_ibt[param].fillna(0)
    
### PREPARING DATA
X_train = np.array(list(ds_ibt[input_variables].data_vars.values()))
X_train = np.reshape(X_train, (X_train.shape[1], X_train.shape[0], X_train.shape[2])) # shape (n_storms, n_features, date_time) = (115, 4, 360)
y_train = np.array(list(ds_ibt[target_variable].data_vars.values()))
y_train = np.reshape(y_train, (y_train.shape[1], y_train.shape[0], y_train.shape[2])) # shape (n_storms, n_features, date_time) = (115, 1, 360)
print('X shape:', X_train.shape)
print('y shape:', y_train.shape)

X shape: (115, 4, 360)
y shape: (115, 1, 360)


In [8]:
class CompleteTimeseriesDataset(Dataset):   
    '''
    Complete Timeseries Dataset.
    Checks that usa_rmw is contains at least a 5-days sequence of non NaN values.
    In that case, fills the NaN values with 0 padding, for every variable.
    '''
    def __init__(self, X: np.ndarray, y: np.ndarray,  device: str='cpu'):
        self.X          = torch.tensor(X).float()
        self.y          = torch.tensor(y).float()
        self.device     = device

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        # Select data & Move to GPU if available
        input_tensor  = self.X[i, :, :].to(self.device)
        target_tensor = self.y[i, :, :].to(self.device)
        return input_tensor, target_tensor



In [9]:
train_dataset = CompleteTimeseriesDataset(X_train, 
                                          y_train,
                                          device
                                         )

train_loader = DataLoader(train_dataset, batch_size=PARAMS['batch_size'], shuffle=True)
X, y = next(iter(train_loader)) # Check
print("Features shape (batch_size, seq_len, n_features):", X.shape)
print("Target shape (batch_size, n_features):", y.shape)

Features shape (batch_size, seq_len, n_features): torch.Size([8, 4, 360])
Target shape (batch_size, n_features): torch.Size([8, 1, 360])


In [10]:
# LR_variables = ['usa_lon', 'usa_lat', 'usa_r34']
# ds_ibt_LR    = ds_ibt[LR_variables]
# ds_ibt_LR

In [11]:
# ### CREATE DATASET (Train and test)
# # FIRST WE AVERAGE OVER EVERY QUADRANT
# # Pre-processing
# MU    = {}
# SIG   = {}
# SCALE = {'usa_wind': 3,
#          'usa_rmw':  3, 
#          'usa_r34':  2,
#          'usa_lon':  0.5,
#          'usa_lat':  1,
# }
# if PARAMS['feature_scaling']:
#     for param in params_of_interest:
#         MU[param]     = float(ds_ibt[param].mean(skipna=True))
#         SIG[param]    = float(ds_ibt[param].std(skipna=True))
#         ds_ibt[param] = SCALE[param] * ((ds_ibt[param] - MU[param]) / SIG[param])

# # Separate train and test set
# sep1 = int(0.6 * len(ds_ibt['storm'])) # 60% train, 20% valid, 20% test
# sep2 = int(0.8 * len(ds_ibt['storm']))
# ds_train, ds_valid, ds_test = ds_ibt.isel(storm=slice(None, sep1)), ds_ibt.isel(storm=slice(sep1, sep2)),  ds_ibt.isel(storm=slice(sep2, None))
# print('Train set: %i storms;  '%len(ds_train['storm']), 'Valid set: %i storms'%len(ds_valid['storm']), 'Test set: %i storms'%len(ds_test['storm']))

# # Create Dataset
# X_train, y_train = du.create_dataset(ds_train, params_of_interest, PARAMS)
# X_valid, y_valid = du.create_dataset(ds_valid, params_of_interest, PARAMS)
# X_test, y_test   = du.create_dataset(ds_test,  params_of_interest, PARAMS)
# print('Shape of predictors matrix X_train: ', np.asarray(X_train).shape)
# print('Shape of targets matrix y_train: ', np.asarray(y_train).shape)

In [12]:
# Open Datasets
# Train, valid, and test sets are moved to GPU if available
train_dataset = pu.ShortTimeseriesDataset(X_train, 
                                          y_train,
                                          PARAMS['n_features'],
                                          PARAMS['seq_len'],
                                          device
                                         )
valid_dataset = pu.ShortTimeseriesDataset(X_valid, 
                                          y_valid,
                                          PARAMS['n_features'],
                                          PARAMS['seq_len'],
                                          device
                                         )
test_dataset  = pu.ShortTimeseriesDataset(X_test, 
                                          y_test,
                                          PARAMS['n_features'],
                                          PARAMS['seq_len'],
                                          device
                                         )

# DataLoader
# torch.manual_seed(99)
train_loader = DataLoader(train_dataset, batch_size=PARAMS['batch_size'], shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=PARAMS['batch_size'], shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=PARAMS['batch_size'], shuffle=False)

X, y = next(iter(train_loader)) # Check
print("Features shape (batch_size, seq_len, n_features):", X.shape)
print("Target shape (batch_size, n_features):", y.shape)

NameError: name 'X_valid' is not defined

In [ ]:
# Declare model
# Normally, LSTM1 handles batch_size
model = pu.LSTM1(num_classes=PARAMS['n_features'], 
                 input_size=PARAMS['input_size'],
                 hidden_size=PARAMS['hidden_size'],
                 num_layers=PARAMS['num_layers'],
                 seq_len=PARAMS['seq_len'],
                 dropout=PARAMS['dropout'],
)
print(model)
print('Trainable parameters:', sum(p.numel() for p in model.parameters() if p.requires_grad))
# Move to GPU if available
model.to(device)

# Loss function, optimizer
loss_function = torch.nn.MSELoss()    # mean-squared error for regression
optimizer     = torch.optim.Adam(model.parameters(), lr=PARAMS['learn_rate']) 

In [ ]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss  = 0
    model.train()

    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    # print(f"Train loss: {avg_loss}")
    return avg_loss

def valid_model(data_loader, model, loss_function):
    num_batches = len(data_loader)
    total_loss  = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    # print(f"Valid loss: {avg_loss}")
    return avg_loss
    
def test_model(data_loader, model, loss_function):
    num_batches = len(data_loader)
    total_loss  = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    # print(f"Test loss: {avg_loss}")
    return avg_loss

def predict(data_loader, model):
    '''
    CAVEAT: Model is put onto CPU in this function.
    '''
    output = torch.tensor([])
    
    model.cpu()
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)

    return output

print("Untrained valid loss\n--------")
best_vl = valid_model(valid_loader, model, loss_function)
print(best_vl)
print()

train_loss = []
valid_loss = []
test_loss  = []
for ix_epoch in tqdm(range(PARAMS['n_epochs'])):
    tl = train_model(train_loader, model, loss_function, optimizer=optimizer)
    vl = valid_model(valid_loader, model, loss_function)
    train_loss.append(tl)
    valid_loss.append(vl)
    test_loss.append(test_model(test_loader, model, loss_function))
    if ix_epoch % (PARAMS['n_epochs'] // 4) == 0:
        print(f"Epoch {ix_epoch} / {PARAMS['n_epochs']}\n---------")
        print(f"Train loss: {tl}\nValid loss: {vl}\n")
    # Save model
    if vl < best_vl:
        torch.save(model.state_dict(), PATHS['lstm_path'])

# Plot
plt.plot(train_loss, label='Train loss', color='tab:blue')
plt.plot(valid_loss, label='Valid loss', color='tab:red')
plt.plot(test_loss,  label='Test loss',  color='tab:olive')
plt.xlabel('Epoch');plt.ylabel('Loss')
plt.legend(loc='lower left');plt.grid()

# Load best model
model.load_state_dict(torch.load(PATHS['lstm_path']))

In [ ]:
### METRICS
# Linear regression
lrg = linear_model.LinearRegression()
lrg.fit(X_train, y_train)
print('==> LINEAR \nR2: ', lrg.score(X_test, y_test))
print('RMSE Vmax (m/s): ', round(du.rmse(du.inverse_scale_normalize(lrg.predict(X_test)[:, 2], MU, SIG, SCALE, 'usa_wind'), du.inverse_scale_normalize(np.asarray(y_test)[:, 2], MU, SIG, SCALE, 'usa_wind')), 2))
print('RMSE R34  (km) : ', round(du.rmse(du.inverse_scale_normalize(lrg.predict(X_test)[:, 3], MU, SIG, SCALE, 'usa_r34'), du.inverse_scale_normalize(np.asarray(y_test)[:, 3], MU, SIG, SCALE, 'usa_r34')), 2))
print('RMSE Rmax (km) : ', round(du.rmse(du.inverse_scale_normalize(lrg.predict(X_test)[:, 4], MU, SIG, SCALE, 'usa_rmw'), du.inverse_scale_normalize(np.asarray(y_test)[:, 4], MU, SIG, SCALE, 'usa_rmw')), 2))

# LSTM
test_dataset_cpu = pu.ShortTimeseriesDataset(X_test, y_test, PARAMS['n_features'], PARAMS['seq_len'])
test_loader_cpu  = DataLoader(test_dataset_cpu,  batch_size=PARAMS['batch_size'], shuffle=False)
preds            = predict(test_loader_cpu, model).cpu().detach().numpy()
print('\n==> LSTM')
print('RMSE Vmax (m/s): ', round(du.rmse(du.inverse_scale_normalize(preds[:, 2], MU, SIG, SCALE, 'usa_wind'), du.inverse_scale_normalize(np.asarray(y_test)[:, 2], MU, SIG, SCALE, 'usa_wind')), 2))
print('RMSE R34  (km) : ', round(du.rmse(du.inverse_scale_normalize(preds[:, 3], MU, SIG, SCALE, 'usa_r34'), du.inverse_scale_normalize(np.asarray(y_test)[:, 3], MU, SIG, SCALE, 'usa_r34')), 2))
print('RMSE Rmax (km) : ', round(du.rmse(du.inverse_scale_normalize(preds[:, 4], MU, SIG, SCALE, 'usa_rmw'), du.inverse_scale_normalize(np.asarray(y_test)[:, 4], MU, SIG, SCALE, 'usa_rmw')), 2))

In [ ]:
# Choose test sample
ds_im        = ds_test.isel(storm=slice(6, 7))

# Prepare inference data
X_im, y_im   = du.create_dataset(ds_im,  params_of_interest, PARAMS)
im_dataset   = pu.ShortTimeseriesDataset(X_im, 
                                          y_im,
                                          PARAMS['n_features'],
                                          PARAMS['seq_len'],
                                         )
im_loader    = DataLoader(im_dataset,  batch_size=PARAMS['batch_size'], shuffle=False)
# Inference
y_pred       = predict(im_loader, model)
# Target
y_im         = np.array(y_im)

# Plot
for i, param in enumerate(params_of_interest):
    feature = y_pred[:, i].cpu().detach().numpy()
    feature = du.inverse_scale_normalize(feature, MU, SIG, SCALE, param)
    target  = y_im[:, i]
    target  = du.inverse_scale_normalize(target,  MU, SIG, SCALE, param)
    lin_fea = du.inverse_scale_normalize(lrg.predict(X_im)[:, i], MU, SIG, SCALE, param)
    plt.title(param, weight='bold')
    plt.plot(target,  label='Data', linewidth=3, color='tab:blue')
    plt.plot(feature, label='LSTM', color='tab:pink')
    plt.plot(lin_fea, label='Regression', color='tab:green', linestyle=':')
    plt.legend(loc='upper left');plt.grid()
    plt.show()